<a href="https://colab.research.google.com/github/a-winders/MLA_ATW/blob/main/MLA2_ATW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Assignment 2
# Adam Winders 
# CS430-ON 
# Machine Learning in the Cloud

<br>
<br>
<br>

## Standard Imports

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Imports for specific models
In this section, you will import the models from SKLearn that you will use this in this notebook. <br> <br>
**Include imports for both Linear Regression and Logistic Regression**

In [39]:
# Imports LinearRegression & LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

Linear Regression - Acquire data

In [38]:
df = pd.read_csv('/content/gdrive/MyDrive/Bellarmine/CS430_Data_Files/Steel_industry_data.csv')

Rename the columns according to **Table 1: Columns Names**

In [28]:
# displays old column names with first 5 rows
df.head()

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [44]:
# renames columns
df.rename(columns=
          {'Usage_kWh':'usage_kwh'
          ,'Lagging_Current_Reactive.Power_kVarh':'lag_react_pwr_kvarh'
          ,'Leading_Current_Reactive_Power_kVarh':'lead_react_pwr_kvarh'
          ,'Lagging_Current_Power_Factor':'lag_current_pwr'
          ,'Leading_Current_Power_Factor':'lead_current_pwr'
          ,'NSM':'nsm'
          ,'WeekStatus':'week_status'
          ,'Day_of_week':'day_of_week'
          ,'Load_Type':'load_type'
           }, inplace = True)


In [45]:
# displays renamed columns with first 5 rows
df.head()

,date,usage_kwh,lag_react_pwr_kvarh,lead_react_pwr_kvarh,CO2(tCO2),lag_current_pwr,lead_current_pwr,nsm,week_status,day_of_week,load_type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


## Imports for Splitting Data and Building Pipeline


In [40]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display='diagram')

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

## Imports for Performance Measures

In [22]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

## Confusion Matrix Function
This function is for creating a graphical version of the confusion matrix.



In [41]:
def plot_cm(y_test, y_pred):
    cm = confusion_matrix(y_test,y_pred)
    fig = plt.figure(figsize=(10,10))
    heatmap = sns.heatmap(cm, annot=True, fmt='.2f', cmap='RdYlGn')
    plt.ylabel('True label')
    plt.xlabel('Predicted Label')

## ROC_Curve Function
This function is for plotting the roc_curve

In [42]:
def plot_roc_curve(fpr,tpr):
  plt.plot(fpr, tpr, linewidth=2)
  plt.plot([0,1],[0,1], 'k--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.0])

##Statistics and Visual Exploration

Summarize the numerical columns of data

In [50]:
df.describe()

,usage_kwh,lag_react_pwr_kvarh,lead_react_pwr_kvarh,CO2(tCO2),lag_current_pwr,lead_current_pwr,nsm
count,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000
mean,27.386892,13.035384,3.870949,0.011524,80.578056,84.367870,42750.000000
std,33.444380,16.306000,7.424463,0.016151,18.921322,30.456535,24940.534317
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.200000,2.300000,0.000000,0.000000,63.320000,99.700000,21375.000000
50%,4.570000,5.000000,0.000000,0.000000,87.960000,100.000000,42750.000000
75%,51.237500,22.640000,2.090000,0.020000,99.022500,100.000000,64125.000000
max,157.180000,96.910000,27.760000,0.070000,100.000000,100.000000,85500.000000


Get the counts of the different load types. Save this into a data frame so you can plot it.

Create a barplot to view how many days fall under each load type. The size of the plot should be 10 x 5.

Create a pairplot of all of the numerical variables using Seaborn

In a markdown comment - write up any interesting patterns you see from the pairplot

## Splitting Data into Train/Test

Create a data frame called X of your inputs that includes all columns except data and usage_kwh

Create a data frame called y of your output that includes only usage_kwh. This will be your target variable that you are trying to predict.

Split the data into Train/Test sets. Use 20% of the data for testing. You should have 4 data frames: **X_train, X_test, y_train, y_test**

## Building the Pipeline

Create a column transformer that will OneHotEncode the categorical variables and MinMax scale the numerical variables (not the output)

Create a pipeline that will transform the data and then run a Linear Regression

## Executing the Model

Run the pipeline to fit the Linear Regression Model

## Evaluating the Model

Run the pipeline to predict y values from the test data set and save them in a variable called **y_pred**

Calcuate the root mean squared error for both train and test. In a markdown comment below the result, interpret the output you receive.

Calculate the R-squared value of the model (test only). In a markdown comment below the result, interpret the output you receive.

Write the code to display the intercept value (bias) and coefficients of the model. **(These will be two different lines of code).**

## Conclusion

In a markdown comment, write a conclusion on how suitable you think this model is for predicting kwh usage.